# Installations

In [1]:
install = True
run_test = True

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
install= True
if install == True:
  !pip install scikit-learn==0.24.2

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import set_config; set_config(display='diagram')
import xgboost as xgb


In [5]:
install = True
if install == True:
  from scipy import stats
  from sklearn.model_selection import train_test_split
  from sklearn.preprocessing import MinMaxScaler, StandardScaler
  from sklearn.pipeline import Pipeline, make_pipeline
  from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
  from sklearn.impute import SimpleImputer, KNNImputer
  from sklearn.preprocessing import RobustScaler, OneHotEncoder, OrdinalEncoder
  from sklearn.metrics import make_scorer
  from sklearn.model_selection import cross_val_score
  from sklearn.neighbors import KNeighborsRegressor
  from sklearn.linear_model import Ridge, Lasso, LinearRegression
  from sklearn.model_selection import RandomizedSearchCV
  from sklearn.svm import SVR
  from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
  from sklearn.ensemble import AdaBoostRegressor
  from sklearn.tree import DecisionTreeRegressor
  from sklearn.ensemble import VotingRegressor
  from sklearn.ensemble import GradientBoostingRegressor
  from sklearn.ensemble import StackingRegressor
  from sklearn.ensemble import RandomForestRegressor

# Demande du client

> How to predict our CA ?

# Donnée de départs

In [8]:
dataset_d16 = '../raw_data/preproc_data_d16.csv'
dataset_d2 = '../raw_data/preproc_data_d2.csv'
data_d16 = pd.read_csv(dataset_d16)
data_d2 = pd.read_csv(dataset_d2)
data_d16.head()

,index,date,jour,service,CA_TTC,CA_HT,TVA,temp,feels_like,temp_min,...,wind_speed,clouds_all,weather_main,weather_description,match_edf,roland_garros,fashion_week,Match Happening,Match Happening-CL,vacances_paris
0,0,2019-08-27,Mardi,midi,0.00,0.00,0.00,303.84,303.95,302.16,...,1.00,0,Clear,sky is clear,0.0,0.0,0.0,NaN,NaN,1
1,1,2019-08-27,Mardi,soir,59.50,51.70,7.80,302.00,301.61,297.23,...,0.89,0,Clear,sky is clear,0.0,0.0,0.0,NaN,NaN,1
2,2,2019-08-28,Mercredi,midi,0.00,0.00,0.00,301.95,302.12,300.75,...,1.79,0,Clear,sky is clear,0.0,0.0,0.0,NaN,NaN,1
3,3,2019-08-28,Mercredi,soir,1723.75,1530.14,193.61,297.94,298.09,295.40,...,1.34,0,Clear,sky is clear,0.0,0.0,0.0,NaN,NaN,1
4,4,2019-08-29,Jeudi,midi,0.00,0.00,0.00,294.88,294.73,293.25,...,2.10,90,Clouds,overcast clouds,0.0,0.0,0.0,NaN,NaN,1


In [9]:
data_d16['Match Happening'].value_counts()

0.0    49
Name: Match Happening, dtype: int64

In [10]:
data_d16['Match Happening'].replace(to_replace=0.0, value = 1, inplace = True)
data_d2['Match Happening'].replace(to_replace=0.0, value = 1, inplace = True)
data_d16['Match Happening-CL'].replace(to_replace=0.0, value = 1, inplace = True)
data_d2['Match Happening-CL'].replace(to_replace=0.0, value = 1, inplace = True)

In [11]:
data_d16.fillna(value = 0, inplace = True)
data_d2.fillna(value = 0, inplace = True)
data_d16

,index,date,jour,service,CA_TTC,CA_HT,TVA,temp,feels_like,temp_min,...,wind_speed,clouds_all,weather_main,weather_description,match_edf,roland_garros,fashion_week,Match Happening,Match Happening-CL,vacances_paris
0,0,2019-08-27,Mardi,midi,0.00,0.00,0.00,303.84,303.95,302.16,...,1.00,0,Clear,sky is clear,0.0,0.0,0.0,0.0,0.0,1
1,1,2019-08-27,Mardi,soir,59.50,51.70,7.80,302.00,301.61,297.23,...,0.89,0,Clear,sky is clear,0.0,0.0,0.0,0.0,0.0,1
2,2,2019-08-28,Mercredi,midi,0.00,0.00,0.00,301.95,302.12,300.75,...,1.79,0,Clear,sky is clear,0.0,0.0,0.0,0.0,0.0,1
3,3,2019-08-28,Mercredi,soir,1723.75,1530.14,193.61,297.94,298.09,295.40,...,1.34,0,Clear,sky is clear,0.0,0.0,0.0,0.0,0.0,1
4,4,2019-08-29,Jeudi,midi,0.00,0.00,0.00,294.88,294.73,293.25,...,2.10,90,Clouds,overcast clouds,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
841,872,2021-07-29,Jeudi,soir,6564.00,5842.95,721.05,295.25,294.82,293.58,...,0.89,0,Clear,sky is clear,0.0,0.0,0.0,0.0,0.0,1
842,873,2021-07-30,Vendredi,midi,1361.00,1229.24,131.76,295.70,295.29,294.40,...,2.24,40,Clouds,scattered clouds,0.0,0.0,0.0,0.0,0.0,1
843,874,2021-07-30,Vendredi,soir,7058.00,6263.03,794.97,294.44,294.01,293.27,...,1.34,0,Clear,sky is clear,0.0,0.0,0.0,0.0,0.0,1
844,875,2021-07-31,Samedi,midi,1655.50,1496.52,158.98,294.69,294.47,294.16,...,0.45,0,Clear,sky is clear,0.0,0.0,0.0,0.0,0.0,1


In [12]:
data_d16.vacances_paris.replace(to_replace = '0', value = 0, inplace=True)
data_d2.vacances_paris.replace(to_replace = '0', value = 0, inplace=True)

In [13]:
data_d16.drop(columns = 'vacances_paris', inplace = True)
data_d2.drop(columns = 'vacances_paris', inplace = True)

In [14]:
data_d16.drop(columns = ['CA_HT','TVA'], inplace = True)
data_d2.drop(columns = ['CA_HT','TVA'], inplace = True)

In [15]:
data_d16.isnull().sum()

index                  0
date                   0
jour                   0
service                0
CA_TTC                 0
temp                   0
feels_like             0
temp_min               0
temp_max               0
wind_speed             0
clouds_all             0
weather_main           0
weather_description    0
match_edf              0
roland_garros          0
fashion_week           0
Match Happening        0
Match Happening-CL     0
dtype: int64

In [16]:
weather_main = pd.DataFrame(np.concatenate([data_d16.weather_main.unique(),data_d2.weather_main.unique()])).drop_duplicates()
weather_main.sort_values(by=0,axis=0, inplace=True)
weather_main

,0
0,Clear
1,Clouds
3,Drizzle
5,Fog
7,Haze
4,Mist
2,Rain
13,Snow
6,Thunderstorm


In [17]:
weather_description = pd.DataFrame(np.concatenate([data_d16.weather_description.unique(),data_d2.weather_description.unique()])).drop_duplicates()
weather_description.sort_values(by=0,axis=0, inplace=True)
weather_description

,0
4,broken clouds
7,drizzle
3,few clouds
14,fog
17,haze
13,heavy intensity drizzle
16,heavy intensity rain
10,light intensity drizzle
11,light intensity drizzle rain
9,light intensity shower rain


In [18]:
pd.DataFrame(weather_description).nunique()

0    22
dtype: int64

In [19]:
weather_main

,0
0,Clear
1,Clouds
3,Drizzle
5,Fog
7,Haze
4,Mist
2,Rain
13,Snow
6,Thunderstorm


In [20]:
weather_description.reset_index().drop(columns=['index'])

,0
0,broken clouds
1,drizzle
2,few clouds
3,fog
4,haze
5,heavy intensity drizzle
6,heavy intensity rain
7,light intensity drizzle
8,light intensity drizzle rain
9,light intensity shower rain


In [21]:
weather_sorted = {
'Clear' : ['sky is clear', np.nan, np.nan,np.nan], 
'Clouds' : ['scattered clouds','few clouds','broken clouds','overcast clouds'],
'Drizzle' : ['light intensity drizzle','drizzle','heavy intensity drizzle',np.nan], 
'Drizzle and Rain' : ['light intensity drizzle rain','rain and drizzle',np.nan,np.nan],
'Fog' : ['fog', np.nan, np.nan,np.nan], 
'Haze' : ['haze', np.nan, np.nan,np.nan], 
'Mist' : ['mist', np.nan, np.nan,np.nan], 
'Rain' : ['light rain','light intensity shower rain','moderate rain', 'heavy intensity rain'], 
'Snow' : ['light snow', np.nan, np.nan,np.nan], 
'Thunderstorm' : ['proximity thunderstorm', 'thunderstorm', 'thunderstorm with light rain','thunderstorm with heavy rain'] 
}

weather_sorted_df = pd.DataFrame(data=weather_sorted)
weather_sorted_df

,Clear,Clouds,Drizzle,Drizzle and Rain,Fog,Haze,Mist,Rain,Snow,Thunderstorm
0,sky is clear,scattered clouds,light intensity drizzle,light intensity drizzle rain,fog,haze,mist,light rain,light snow,proximity thunderstorm
1,NaN,few clouds,drizzle,rain and drizzle,NaN,NaN,NaN,light intensity shower rain,NaN,thunderstorm
2,NaN,broken clouds,heavy intensity drizzle,NaN,NaN,NaN,NaN,moderate rain,NaN,thunderstorm with light rain
3,NaN,overcast clouds,NaN,NaN,NaN,NaN,NaN,heavy intensity rain,NaN,thunderstorm with heavy rain


In [22]:
#data_d16.weather_description.unique()

In [23]:
#weather_sorted_df

In [24]:
#data_d16.columns

In [25]:
#weather_sorted.keys()

In [26]:
for i in weather_sorted.values():
  print(i)

['sky is clear', nan, nan, nan]
['scattered clouds', 'few clouds', 'broken clouds', 'overcast clouds']
['light intensity drizzle', 'drizzle', 'heavy intensity drizzle', nan]
['light intensity drizzle rain', 'rain and drizzle', nan, nan]
['fog', nan, nan, nan]
['haze', nan, nan, nan]
['mist', nan, nan, nan]
['light rain', 'light intensity shower rain', 'moderate rain', 'heavy intensity rain']
['light snow', nan, nan, nan]
['proximity thunderstorm', 'thunderstorm', 'thunderstorm with light rain', 'thunderstorm with heavy rain']


In [27]:
def weather_cat(x, list_values):
  for values in list_values:
    if x == values:
      return values

In [28]:
#x = data_test.reset_index()
#x = x.loc[0]
#print(x)

In [29]:
#set(data_test.weather_description)

In [30]:
run_test = False
if run_test == True:
  data_test = data_d16.copy()
  for i in range(len(list(weather_sorted.values()))):
    nom_col = list(weather_sorted.keys())[i]
    list_col = list(weather_sorted.values())[i]
    data_test[nom_col] = data_test.apply(lambda x: weather_cat(x['weather_description'], list_col), axis=1)


In [31]:
for i in range(len(list(weather_sorted.values()))):
  nom_col = list(weather_sorted.keys())[i]
  list_col = list(weather_sorted.values())[i]
  data_d16[nom_col] = data_d16.apply(lambda x: weather_cat(x['weather_description'], list_col), axis=1)

In [32]:
for i in range(len(list(weather_sorted.values()))):
  nom_col = list(weather_sorted.keys())[i]
  list_col = list(weather_sorted.values())[i]
  data_d2[nom_col] = data_d2.apply(lambda x: weather_cat(x['weather_description'], list_col), axis=1)

In [33]:
data_d16.fillna(value=0, inplace=True)
data_d2.fillna(value=0, inplace=True)

In [34]:
cd '/content/drive/MyDrive/resto-655-project/resto-project-data/Data/Data_Daroco'

[Errno 2] No such file or directory: '/content/drive/MyDrive/resto-655-project/resto-project-data/Data/Data_Daroco'
/Users/monmac/code/tomaymerich14/resto-project/notebooks


In [35]:
'match_edf', 'roland_garros', 'fashion_week'

('match_edf', 'roland_garros', 'fashion_week')

In [36]:
data_d2['Match Happening'] = data_d2['Match Happening'].astype('object')
data_d2['Match Happening-CL'] = data_d2['Match Happening-CL'].astype('object')
data_d2['match_edf'] = data_d2['match_edf'].astype('object')
data_d2['roland_garros'] = data_d2['roland_garros'].astype('object')
data_d2['fashion_week'] = data_d2['fashion_week'].astype('object')

In [37]:
data_d16['Match Happening'] = data_d16['Match Happening'].astype('object')
data_d16['Match Happening-CL'] = data_d16['Match Happening-CL'].astype('object')
data_d16['match_edf'] = data_d16['match_edf'].astype('object')
data_d16['roland_garros'] = data_d16['roland_garros'].astype('object')
data_d16['fashion_week'] = data_d16['fashion_week'].astype('object')

In [38]:
data_d2

,index,date,jour,service,CA_TTC,temp,feels_like,temp_min,temp_max,wind_speed,...,Clear,Clouds,Drizzle,Drizzle and Rain,Fog,Haze,Mist,Rain,Snow,Thunderstorm
0,0,2019-01-01,Mardi,soir,8345.0,280.87,280.87,279.48,281.45,0.45,...,0,0,light intensity drizzle,0,0,0,0,0,0,0
1,1,2019-01-02,Mercredi,midi,3939.0,279.57,278.92,278.83,280.31,1.34,...,0,overcast clouds,0,0,0,0,0,0,0,0
2,2,2019-01-02,Mercredi,soir,9045.5,278.95,277.22,277.71,279.81,2.24,...,0,overcast clouds,0,0,0,0,0,0,0,0
3,3,2019-01-03,Jeudi,midi,4836.0,279.11,279.11,278.06,280.25,0.89,...,0,overcast clouds,0,0,0,0,0,0,0,0
4,4,2019-01-03,Jeudi,soir,10024.0,278.57,278.57,277.38,279.69,0.45,...,0,overcast clouds,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1318,1369,2021-07-29,Jeudi,soir,7230.0,295.25,294.82,293.58,296.82,0.89,...,sky is clear,0,0,0,0,0,0,0,0,0
1319,1370,2021-07-30,Vendredi,midi,2104.0,295.70,295.29,294.40,297.38,2.24,...,0,scattered clouds,0,0,0,0,0,0,0,0
1320,1371,2021-07-30,Vendredi,soir,9816.5,294.44,294.01,293.27,295.71,1.34,...,sky is clear,0,0,0,0,0,0,0,0,0
1321,1372,2021-07-31,Samedi,midi,1202.5,294.69,294.47,294.16,295.59,0.45,...,sky is clear,0,0,0,0,0,0,0,0,0


In [39]:
#data_d16.to_csv(path_or_buf='data_d16.csv', index= False)
#data_d2.to_csv(path_or_buf='data_d2.csv', index= False)

In [40]:
#data_test

In [41]:
#list(set(data_test.Clouds))

In [42]:
#list(set(data_test.Clouds))

In [43]:
#data_test.keys()

# Preprocessing

In [44]:
data_d16.drop(columns=['Snow','Haze'], inplace=True)
data_d2.drop(columns=['Snow','Haze'], inplace=True)

In [45]:
dataTypeSeries = data_d16.dtypes
dataTypeSeries_valuecounts = data_d16.dtypes.value_counts()
print(dataTypeSeries_valuecounts)

object     18
float64     6
int64       2
dtype: int64


In [46]:
feat_numerical_nunique_d2 = pd.DataFrame(data_d2.select_dtypes(exclude=['object'],include=['int64','float64']).nunique(), columns = ["unique_values"])
feat_numerical_nunique_d2

,unique_values
index,1323
CA_TTC,1297
temp,1040
feels_like,1081
temp_min,605
temp_max,676
wind_speed,43
clouds_all,7


In [47]:
feat_categorical_nunique_d2 = pd.DataFrame(data_d2.select_dtypes(include=['object'],exclude=['int64','float64']).nunique(), columns = ["unique_values"])
feat_categorical_nunique_d2

,unique_values
date,672
jour,7
service,2
weather_main,8
weather_description,21
match_edf,2
roland_garros,2
fashion_week,2
Match Happening,2
Match Happening-CL,2


In [48]:
feat_numerical_nunique_d16 = pd.DataFrame(data_d16.select_dtypes(exclude=['object'],include=['int64','float64']).nunique(), columns = ["unique_values"])
feat_numerical_nunique_d16

,unique_values
index,846
CA_TTC,823
temp,710
feels_like,732
temp_min,448
temp_max,495
wind_speed,43
clouds_all,7


In [49]:
data_d2["Match Happening-CL"].value_counts()

0.0    1307
1.0      16
Name: Match Happening-CL, dtype: int64

In [50]:
feat_categorical_nunique_d16 = pd.DataFrame(data_d16.select_dtypes(include=['object'],exclude=['int64','float64']).nunique(), columns = ["unique_values"])
feat_categorical_nunique_d16

,unique_values
date,432
jour,7
service,2
weather_main,8
weather_description,19
match_edf,2
roland_garros,2
fashion_week,2
Match Happening,2
Match Happening-CL,2


In [51]:
#data_d16.select_dtypes(include=['object'],exclude=['int64','float64']).nunique().index.sort_values

In [52]:
#weather_sorted

In [53]:
#data_test_d2.Snow.unique()

In [54]:
run_test=False
if run_test==True:
  #data_test_d16 = data_d16.copy()
  #data_test_d2 = data_d2.copy()

  ordinal_features_d2 = ['Clear','Clouds','Drizzle','Drizzle and Rain','Fog','Mist','Rain','Thunderstorm']
  ordinal_features_d16 = ['Clear','Clouds','Drizzle','Drizzle and Rain','Fog','Mist','Rain','Thunderstorm']

  # 10 weather features
  Clear= [0,'sky is clear']
  Clouds= [0,'scattered clouds','few clouds','broken clouds','overcast clouds']
  Drizzle= [0,'light intensity drizzle','drizzle','heavy intensity drizzle']
  Drizzle_and_Rain= [0,'light intensity drizzle rain','rain and drizzle']
  Fog= [0,'fog']
  Haze= [0,'haze']
  Mist= [0,'mist']
  Rain= [0,'light rain','light intensity shower rain','moderate rain','heavy intensity rain']
  Snow= [0,'light snow']
  Thunderstorm= [0,'proximity thunderstorm','thunderstorm','thunderstorm with light rain','thunderstorm with heavy rain']

  # Encoder
  encoder_d2 = OrdinalEncoder(
      categories=[
          Clear,
          Clouds,
          Drizzle,
          Drizzle_and_Rain,
          Fog,
          Mist,
          Rain,
          Thunderstorm
          
      ],
      handle_unknown="use_encoded_value",
      unknown_value=-1
  )

  # Encoder
  encoder_d16 = OrdinalEncoder(
      categories=[
          Clear,
          Clouds,
          Drizzle,
          Drizzle_and_Rain,
          Fog,
          Mist,
          Rain,
          Thunderstorm
          
      ],
      handle_unknown="use_encoded_value",
      unknown_value=-1
  )

In [55]:
#encoder_d2.fit_transform(data_d2[ordinal_features_d2])

In [56]:
#encoder_d16.fit_transform(data_d16[ordinal_features_d16])

In [57]:
feat_ordinal_d16_dict = {
    'Clear': [0,'sky is clear'],
    'Clouds': [0,'scattered clouds','few clouds','broken clouds','overcast clouds'],
    'Drizzle': [0,'light intensity drizzle','drizzle','heavy intensity drizzle'],
    'Drizzle_and_Rain': [0,'light intensity drizzle rain','rain and drizzle'],
    'Fog': [0,'fog'],
    #'Haze': [0,'haze'],
    'Mist': [0,'mist'],
    'Rain': [0,'light rain','light intensity shower rain','moderate rain','heavy intensity rain'],
    #'Snow': [0,'light snow'],
    'Thunderstorm': [0,'proximity thunderstorm','thunderstorm','thunderstorm with light rain','thunderstorm with heavy rain']
}

feat_ordinal_d16 = sorted(feat_ordinal_d16_dict.keys()) # sort alphabetically
feat_ordinal_d16_values_sorted = [feat_ordinal_d16_dict[i] for i in feat_ordinal_d16]

encoder_ordinal_d16 = OrdinalEncoder(
    categories=feat_ordinal_d16_values_sorted,
    dtype= np.int64,
    handle_unknown="use_encoded_value",
    unknown_value=-1 # Considers unknown values as worse than "missing"
)

preproc_ordinal_d16 = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="missing"),
    encoder_ordinal_d16,
    RobustScaler()
)

preproc_ordinal_d16

Pipeline(steps=[('simpleimputer',
                 SimpleImputer(fill_value='missing', strategy='constant')),
                ('ordinalencoder',
                 OrdinalEncoder(categories=[[0, 'sky is clear'],
                                            [0, 'scattered clouds',
                                             'few clouds', 'broken clouds',
                                             'overcast clouds'],
                                            [0, 'light intensity drizzle',
                                             'drizzle',
                                             'heavy intensity drizzle'],
                                            [0, 'light intensity drizzle rain',
                                             'rain and drizzle'],
                                            [0, 'fog'], [0, 'mist'],
                                            [0, 'light rain',
                                             'light intensity shower rain',
                                             'moderate rain',
                                             'heavy intensity rain'],
                                            [0, 'proximity thunderstorm',
                                             'thunderstorm',
                                             'thunderstorm with light rain',
                                             'thunderstorm with heavy rain']],
                                dtype=<class 'numpy.int64'>,
                                handle_unknown='use_encoded_value',
                                unknown_value=-1)),
                ('robustscaler', RobustScaler())])

In [58]:
feat_ordinal_dict = {
    'Clear': [0,'sky is clear'],
    'Clouds': [0,'scattered clouds','few clouds','broken clouds','overcast clouds'],
    'Drizzle': [0,'light intensity drizzle','drizzle','heavy intensity drizzle'],
    'Drizzle_and_Rain': [0,'light intensity drizzle rain','rain and drizzle'],
    'Fog': [0,'fog'],
    'Mist': [0,'mist'],
    'Rain': [0,'light rain','light intensity shower rain','moderate rain','heavy intensity rain'],
    'Thunderstorm': [0,'proximity thunderstorm','thunderstorm','thunderstorm with light rain','thunderstorm with heavy rain']
}

feat_ordinal = sorted(feat_ordinal_dict.keys()) # sort alphabetically
feat_ordinal_values_sorted = [feat_ordinal_dict[i] for i in feat_ordinal]

encoder_ordinal = OrdinalEncoder(
    categories=feat_ordinal_values_sorted,
    dtype= np.int64,
    handle_unknown="use_encoded_value",
    unknown_value=-1 # Considers unknown values as worse than "missing"
)

preproc_ordinal = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="missing"),
    encoder_ordinal,
    RobustScaler()
)

preproc_ordinal

Pipeline(steps=[('simpleimputer',
                 SimpleImputer(fill_value='missing', strategy='constant')),
                ('ordinalencoder',
                 OrdinalEncoder(categories=[[0, 'sky is clear'],
                                            [0, 'scattered clouds',
                                             'few clouds', 'broken clouds',
                                             'overcast clouds'],
                                            [0, 'light intensity drizzle',
                                             'drizzle',
                                             'heavy intensity drizzle'],
                                            [0, 'light intensity drizzle rain',
                                             'rain and drizzle'],
                                            [0, 'fog'], [0, 'mist'],
                                            [0, 'light rain',
                                             'light intensity shower rain',
                                             'moderate rain',
                                             'heavy intensity rain'],
                                            [0, 'proximity thunderstorm',
                                             'thunderstorm',
                                             'thunderstorm with light rain',
                                             'thunderstorm with heavy rain']],
                                dtype=<class 'numpy.int64'>,
                                handle_unknown='use_encoded_value',
                                unknown_value=-1)),
                ('robustscaler', RobustScaler())])

In [59]:
feat_numerical_nunique_d16

,unique_values
index,846
CA_TTC,823
temp,710
feels_like,732
temp_min,448
temp_max,495
wind_speed,43
clouds_all,7


In [60]:
print(list(pd.DataFrame(data_d16.select_dtypes(include=['object'],exclude=['int64','float64']).nunique(), columns = ["unique_values"]).index))
print(list(pd.DataFrame(data_d16.select_dtypes(exclude=['object'],include=['int64','float64']).nunique(), columns = ["unique_values"]).index))

['date', 'jour', 'service', 'weather_main', 'weather_description', 'match_edf', 'roland_garros', 'fashion_week', 'Match Happening', 'Match Happening-CL', 'Clear', 'Clouds', 'Drizzle', 'Drizzle and Rain', 'Fog', 'Mist', 'Rain', 'Thunderstorm']
['index', 'CA_TTC', 'temp', 'feels_like', 'temp_min', 'temp_max', 'wind_speed', 'clouds_all']


In [61]:
data_d16.date.value_counts()

2019-08-27    2
2020-07-15    2
2020-08-25    2
2020-08-07    2
2020-08-06    2
             ..
2020-06-12    1
2019-11-25    1
2021-06-02    1
2020-08-24    1
2020-05-15    1
Name: date, Length: 432, dtype: int64

In [62]:
categorical_features = ['date','jour', 'service', 'weather_main', 'weather_description', 'Match Happening', 'Match Happening-CL', 'match_edf', 'roland_garros', 'fashion_week']
numerical_features = ['temp', 'feels_like', 'temp_min', 'temp_max', 'wind_speed', 'clouds_all']
ordinal_features =['Clear', 'Clouds', 'Drizzle', 'Drizzle and Rain', 'Fog', 'Mist', 'Rain', 'Thunderstorm']

In [63]:
preproc_numerical = make_pipeline(
    KNNImputer(),
    RobustScaler())

preproc_categorical = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="missing"),
    OneHotEncoder(handle_unknown="ignore"))

preproc = make_column_transformer(
    (preproc_numerical, numerical_features),
    (preproc_categorical, categorical_features),
    (preproc_ordinal, ordinal_features),
    remainder ="drop")

In [64]:
preproc

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('knnimputer', KNNImputer()),
                                                 ('robustscaler',
                                                  RobustScaler())]),
                                 ['temp', 'feels_like', 'temp_min', 'temp_max',
                                  'wind_speed', 'clouds_all']),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='igno...
                                                                              'moderate '
                                                                              'rain',
                                                                              'heavy '
                                                                              'intensity '
                                                                              'rain'],
                                                                             [0,
                                                                              'proximity '
                                                                              'thunderstorm',
                                                                              'thunderstorm',
                                                                              'thunderstorm '
                                                                              'with '
                                                                              'light '
                                                                              'rain',
                                                                              'thunderstorm '
                                                                              'with '
                                                                              'heavy '
                                                                              'rain']],
                                                                 dtype=<class 'numpy.int64'>,
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1)),
                                                 ('robustscaler',
                                                  RobustScaler())]),
                                 ['Clear', 'Clouds', 'Drizzle',
                                  'Drizzle and Rain', 'Fog', 'Mist', 'Rain',
                                  'Thunderstorm'])])

In [65]:
pipe = make_pipeline(
    preproc,
)

In [66]:
pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('knnimputer',
                                                                   KNNImputer()),
                                                                  ('robustscaler',
                                                                   RobustScaler())]),
                                                  ['temp', 'feels_like',
                                                   'temp_min', 'temp_max',
                                                   'wind_speed',
                                                   'clouds_all']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehotencode...
                                                                                               'moderate '
                                                                                               'rain',
                                                                                               'heavy '
                                                                                               'intensity '
                                                                                               'rain'],
                                                                                              [0,
                                                                                               'proximity '
                                                                                               'thunderstorm',
                                                                                               'thunderstorm',
                                                                                               'thunderstorm '
                                                                                               'with '
                                                                                               'light '
                                                                                               'rain',
                                                                                               'thunderstorm '
                                                                                               'with '
                                                                                               'heavy '
                                                                                               'rain']],
                                                                                  dtype=<class 'numpy.int64'>,
                                                                                  handle_unknown='use_encoded_value',
                                                                                  unknown_value=-1)),
                                                                  ('robustscaler',
                                                                   RobustScaler())]),
                                                  ['Clear', 'Clouds', 'Drizzle',
                                                   'Drizzle and Rain', 'Fog',
                                                   'Mist', 'Rain',
                                                   'Thunderstorm'])]))])

# Model

In [67]:
data_d16

,index,date,jour,service,CA_TTC,temp,feels_like,temp_min,temp_max,wind_speed,...,Match Happening,Match Happening-CL,Clear,Clouds,Drizzle,Drizzle and Rain,Fog,Mist,Rain,Thunderstorm
0,0,2019-08-27,Mardi,midi,0.00,303.84,303.95,302.16,305.40,1.00,...,0.0,0.0,sky is clear,0,0,0,0,0,0,0
1,1,2019-08-27,Mardi,soir,59.50,302.00,301.61,297.23,304.58,0.89,...,0.0,0.0,sky is clear,0,0,0,0,0,0,0
2,2,2019-08-28,Mercredi,midi,0.00,301.95,302.12,300.75,302.87,1.79,...,0.0,0.0,sky is clear,0,0,0,0,0,0,0
3,3,2019-08-28,Mercredi,soir,1723.75,297.94,298.09,295.40,300.40,1.34,...,0.0,0.0,sky is clear,0,0,0,0,0,0,0
4,4,2019-08-29,Jeudi,midi,0.00,294.88,294.73,293.25,296.81,2.10,...,0.0,0.0,0,overcast clouds,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
841,872,2021-07-29,Jeudi,soir,6564.00,295.25,294.82,293.58,296.82,0.89,...,0.0,0.0,sky is clear,0,0,0,0,0,0,0
842,873,2021-07-30,Vendredi,midi,1361.00,295.70,295.29,294.40,297.38,2.24,...,0.0,0.0,0,scattered clouds,0,0,0,0,0,0
843,874,2021-07-30,Vendredi,soir,7058.00,294.44,294.01,293.27,295.71,1.34,...,0.0,0.0,sky is clear,0,0,0,0,0,0,0
844,875,2021-07-31,Samedi,midi,1655.50,294.69,294.47,294.16,295.59,0.45,...,0.0,0.0,sky is clear,0,0,0,0,0,0,0


In [68]:
X_d2 = data_d2.drop(columns=["CA_TTC"])
y_d2 = data_d2.CA_TTC
X_d2.head()

,index,date,jour,service,temp,feels_like,temp_min,temp_max,wind_speed,clouds_all,...,Match Happening,Match Happening-CL,Clear,Clouds,Drizzle,Drizzle and Rain,Fog,Mist,Rain,Thunderstorm
0,0,2019-01-01,Mardi,soir,280.87,280.87,279.48,281.45,0.45,90,...,0.0,0.0,0,0,light intensity drizzle,0,0,0,0,0
1,1,2019-01-02,Mercredi,midi,279.57,278.92,278.83,280.31,1.34,90,...,0.0,0.0,0,overcast clouds,0,0,0,0,0,0
2,2,2019-01-02,Mercredi,soir,278.95,277.22,277.71,279.81,2.24,90,...,0.0,0.0,0,overcast clouds,0,0,0,0,0,0
3,3,2019-01-03,Jeudi,midi,279.11,279.11,278.06,280.25,0.89,90,...,0.0,0.0,0,overcast clouds,0,0,0,0,0,0
4,4,2019-01-03,Jeudi,soir,278.57,278.57,277.38,279.69,0.45,90,...,0.0,0.0,0,overcast clouds,0,0,0,0,0,0


In [69]:
X_d2.tail()

,index,date,jour,service,temp,feels_like,temp_min,temp_max,wind_speed,clouds_all,...,Match Happening,Match Happening-CL,Clear,Clouds,Drizzle,Drizzle and Rain,Fog,Mist,Rain,Thunderstorm
1318,1369,2021-07-29,Jeudi,soir,295.25,294.82,293.58,296.82,0.89,0,...,0.0,0.0,sky is clear,0,0,0,0,0,0,0
1319,1370,2021-07-30,Vendredi,midi,295.70,295.29,294.40,297.38,2.24,40,...,0.0,0.0,0,scattered clouds,0,0,0,0,0,0
1320,1371,2021-07-30,Vendredi,soir,294.44,294.01,293.27,295.71,1.34,0,...,0.0,0.0,sky is clear,0,0,0,0,0,0,0
1321,1372,2021-07-31,Samedi,midi,294.69,294.47,294.16,295.59,0.45,0,...,0.0,0.0,sky is clear,0,0,0,0,0,0,0
1322,1373,2021-07-31,Samedi,soir,293.83,293.55,292.49,294.77,0.89,40,...,0.0,0.0,0,scattered clouds,0,0,0,0,0,0


In [70]:
y_d2

0        8345.0
1        3939.0
2        9045.5
3        4836.0
4       10024.0
         ...   
1318     7230.0
1319     2104.0
1320     9816.5
1321     1202.5
1322     8160.5
Name: CA_TTC, Length: 1323, dtype: float64

In [71]:
from sklearn.dummy import DummyRegressor

model_dummy = DummyRegressor(strategy = 'median')
pipe_dummy = make_pipeline(preproc, model_dummy)
cross_val_score(pipe_dummy, X_d2, y_d2, cv=20).mean()

-0.3882767055207269

In [75]:
model_ridge = Ridge()
pipe_ridge = make_pipeline(preproc, model_ridge)
cross_val_score(pipe_ridge, X_d2, y_d2, cv=20).mean()

0.20185861639956681

In [76]:
from xgboost import XGBRegressor

In [82]:
model_xgb = XGBRegressor(max_depth=10, n_estimators=300, learning_rate=0.05)
pipe_xgb = make_pipeline(preproc, model_xgb)
score=cross_val_score(pipe_xgb,X_d2,y_d2,cv=20,n_jobs=-1).mean()

In [83]:
score

0.13526168997668644

In [86]:
model = RandomForestRegressor(max_depth=50,min_samples_leaf=20)
pipe = make_pipeline(preproc, model)
score = cross_val_score(pipe, X_d2, y_d2, cv=20)
print(score.std())
score.mean()

1.2611635063486146


0.24148674911595633

In [92]:
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor(max_depth=50, min_samples_leaf=20)
pipe = make_pipeline(preproc, model)
score = cross_val_score(pipe, X_d2, y_d2, cv=20)
print(score.std())
score.mean()

1.3074615846355822


0.15537504839566

In [88]:
model = AdaBoostRegressor(
    base_estimator=DecisionTreeRegressor(max_depth=None))

pipe = make_pipeline(preproc, model)
score = cross_val_score(pipe, X_d2, y_d2, cv=20)
print(score.std())
score.mean()

1.412343083348557


0.11462583702827192

In [90]:
model = GradientBoostingRegressor(n_estimators=100, verbose=0)
pipe_gb = make_pipeline(preproc, model)
score = cross_val_score(pipe_gb, X_d2, y_d2, cv=20)
print(score.std())
score.mean()

1.2423930852683374


0.21865649803559933

In [91]:
gboost = GradientBoostingRegressor(n_estimators=100)
ridge = Ridge()
svm = SVR(C=1, epsilon=0.05)
adaboost = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=None))


model = VotingRegressor(
    estimators = [("gboost", gboost),("adaboost", adaboost),("ridge", ridge), ("svm_rbf", svm)],
    weights = [1,1,1,1], # to equally weight the two models
    n_jobs=-1
)

pipe_ensemble = make_pipeline(preproc, model)

score = cross_val_score(pipe_ensemble, X_d2, y_d2, cv=20, n_jobs=-1)
print(score.std())
score.mean()

1.0408299679716935


0.2558758132495508

In [ ]:
#pipe_ridge.get_params()

In [ ]:
from sklearn.model_selection import GridSearchCV
run_test = False
if run_test == True:
  grid_search = GridSearchCV(
      pipe, 
      param_grid={'imputer__n_neighbors': [4,5,6]
      },
      cv=5,
      scoring="accuracy")

  grid_search.fit(X, y)
  grid_search.best_params_

In [73]:
run_test = False
if run_test == True:
  tuned_pipe = grid_search.best_estimator_
  tuned_pipe

In [74]:
from sklearn.model_selection import cross_validate
run_test = False
if run_test == True:
  cv_results = cross_validate(tuned_pipe, X,y, cv = 20, scoring = "accuracy")
  base_score = cv_results['test_score'].mean()
  base_score